<a href="https://colab.research.google.com/github/Dominique-Thorendal-Research/momentum_rotation/blob/main/momentum_rotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Momentum ETF Rotation — Learn‑by‑Doing Notebook

This notebook teaches you how to build a **momentum ETF rotation** strategy using `yfinance` for data and `vectorbt` for backtesting.

**Idea:** Every month, pick the **top N** ETFs by past **X‑day** return and hold them equally until next rebalance.

You will learn:
- Data download & cleaning
- Feature engineering (rolling momentum)
- Building target weights for rotation
- Running a backtest with `vectorbt`
- Interpreting stats
- Doing a tiny parameter sweep
- A quick train/test split for sanity


In [ ]:
# If you haven't installed these yet, run this cell once.
# (In Anaconda/VSCode, using your working environment is fine.)
# You can comment out lines that you already have installed.
# Note: Restart kernel after install if imports fail.
!pip install yfinance vectorbt backtesting bt matplotlib pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import vectorbt as vbt

pd.options.display.width = 120
pd.options.display.max_columns = 20


## Configuration

In [ ]:
# Universe: broad, diversified ETFs added later "XLK", "XLF", "XLV", "LQD", "HYG"]
TICKERS = ["SPY", "QQQ", "IWM", "EFA", "TLT", "GLD", "XLK", "XLV"]
START = "2006-01-01"         # multiple regimes
INIT_CASH = 100_000
FEES = 0.0025                 # 10 bps per trade before:0.001
SLIPPAGE = 0.001            # 5 bps orig 0.0005

# Base strategy params (we'll sweep them later)
LOOKBACK_DAYS = 126           # ~3 months = 63
TOP_N = 3                    # hold top 2
REBAL_FREQ = "M"             # rebalance monthly


![image.png](attachment:image.png)

## 1) Download & inspect prices

In [ ]:
px = yf.download(TICKERS, start=START)["Close"].dropna(how="all")
px = px.loc[:, TICKERS].dropna()
px.tail(10)


## 2) Choose rebalance dates

In [ ]:
# Use last trading day of each month
rebal_dates = px.resample(REBAL_FREQ).last().index
rebal_dates[:5], rebal_dates[-5:]


## 3) Momentum feature (rolling total return)

In [ ]:
# Rolling total return over LOOKBACK_DAYS
momentum = px.pct_change(LOOKBACK_DAYS)

# Align momentum to each rebal date (use last known value on that date)
mom_on_rebal = momentum.reindex(rebal_dates, method="ffill").dropna(how="all")
mom_on_rebal.tail()


In [ ]:
#momentum.tail()
momentum.std()

## 4) Build target weights from ranks

### Strategy 1: Orig momentum

In [ ]:
# Orig momentum strategy
def build_rotation_weights(momentum_subset: pd.DataFrame, top_n: int) -> pd.DataFrame:
    """Equal-weight the top_n tickers by momentum at each rebalance date."""
    ranks = momentum_subset.rank(axis=1, ascending=False, method="first")
    select = (ranks <= top_n).astype(float)
    w = select.div(select.sum(axis=1), axis=0).fillna(0.0)

    return w

w_rebal = build_rotation_weights(mom_on_rebal, TOP_N)
w_rebal.tail()




### Strategy 2: Risk Parity

In [ ]:
# Risk Parity
def build_rp_weights(momentum_subset: pd.DataFrame, prices: pd.DataFrame, top_n: int, vol_window: int = 20) -> pd.DataFrame:
    """Risk-parity weighting within top_n winners."""
    ranks = momentum_subset.rank(axis=1, ascending=False, method="first")
    select = (ranks <= top_n).astype(float)

    # Compute rolling vol (annualized optional)
    vol = prices.pct_change().rolling(vol_window).std()
    vol_r = vol.reindex(momentum_subset.index, method="ffill")

    # Inverse-vol weights among selected
    inv_vol = 1 / vol_r
    inv_vol = inv_vol * select  # zero out non-selected assets
    w = inv_vol.div(inv_vol.sum(axis=1), axis=0).fillna(0.0)
    return w

w_rebal = build_rp_weights(mom_on_rebal, px, TOP_N)


### Add Momentum > 0

In [ ]:
composite_mom = (mom_on_rebal * w_rebal).sum(axis=1)
composite_mom.tail()
risk_off = composite_mom < 0
risk_off.tail()
w_defensive = w_rebal.copy()

for date in w_defensive.index:
    if composite_mom.loc[date] < 0:
        # Set all risky weights to 0, invest fully in TLT
        w_defensive.loc[date] = 0
        w_defensive.loc[date, "TLT"] = 1.0
w_defensive


## 5) Expand weights to all trading days

In [ ]:
# If using defensive (momentum <0 -> Treasury)
#w_rebal = w_defensive.copy()

# Forward-fill weights between rebalances
w_all = w_rebal.reindex(px.index).fillna(method="ffill").fillna(0.0)

# Safety re-normalization in case of any row sum drift
w_all = w_all.div(w_all.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)
w_all.tail()


## 6) Backtest with `vectorbt`

In [ ]:
import bt
import ffn

# --- helper: simple commission model (10 bps per notional trade) ---
def bps_commissions(qty, price):
    # qty is shares; notional = |qty| * price
    # 10 bps = 0.001; change to FEES if you want to reuse your variable
    return abs(qty) * price * FEES

# Strategy: every month apply target weights from w_all and rebalance
strat = bt.Strategy(
    'MomentumRotation',
    [
        bt.algos.RunMonthly(),                 # act on monthly schedule
        bt.algos.WeighTarget(w_all),           # use your target weight DF
        bt.algos.Rebalance()                   # trade to targets
    ]
)

bt_test = bt.Backtest(
    strat,
    px,                                       # price DataFrame
    initial_capital=INIT_CASH,
    commissions=bps_commissions,              # optional
    progress_bar=False
)

res = bt.run(bt_test)
res.plot()  # chart (optional)

stats = res.stats
stats.loc[['start', 'end', 'total_return', 'cagr', 'daily_sharpe', 'max_drawdown']]


## 7) Parameter sweep (robustness check)

🧭 7) Parameter Sweep – Finding Robust Strategy Settings

In this step, we test how sensitive our momentum rotation strategy is to different parameter choices:

Lookback period (lookback_days) → how far back we measure momentum (e.g., 2, 3, 4, 6 months).

Top N (top_n) → how many of the best-performing ETFs we hold each month.

For each combination, the code:

Builds momentum signals and target weights using make_rotation_weights().

Runs a full backtest using bt (without commissions for speed and stability).

Collects the daily Sharpe ratio as a performance score.

Stores results in a table (grid_df) that ranks parameter sets from best to worst.

This “parameter sweep” helps you identify which lookback window and number of assets are most robust — instead of overfitting to one arbitrary choice.
You’ll usually want to choose a region of stability (e.g., 63–84 days and Top 2 both work well), not just the single top Sharpe.

In [ ]:
import bt
import pandas as pd
import numpy as np

def make_rotation_weights(px_df: pd.DataFrame, lookback_days: int, top_n: int) -> pd.DataFrame:
    momentum = px_df.pct_change(lookback_days)
    rebal_dates = px_df.resample("M").last().index
    mom_r = momentum.reindex(rebal_dates, method="ffill").dropna(how="all")

    ranks = mom_r.rank(axis=1, ascending=False, method="first")
    select = (ranks <= top_n).astype(float)
    w_rebal = select.div(select.sum(axis=1), axis=0).fillna(0.0)

    # Expand to daily and normalize
    w_all = w_rebal.reindex(px_df.index).ffill().fillna(0.0)
    w_all = w_all.div(w_all.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)
    return w_all

def run_rotation_bt(px_df, lookback_days, top_n):
    w = make_rotation_weights(px_df, lookback_days, top_n)
    strat = bt.Strategy(
        f"Rot_{lookback_days}_{top_n}",
        [bt.algos.RunMonthly(), bt.algos.WeighTarget(w), bt.algos.Rebalance()]
    )
    # ⚠️ commissions=None for stability/speed during sweep
    bt_test = bt.Backtest(strat, px_df, initial_capital=INIT_CASH, progress_bar=False)
    try:
        res = bt.run(bt_test)
        sharpe = float(res.stats.loc['daily_sharpe'])
    except Exception as e:
        # If anything blows up, return NaN so we can see it in the grid
        print(f"Param set failed (lb={lookback_days}, top_n={top_n}): {e}")
        sharpe = np.nan
    return sharpe

grid = []
for lb in [42, 63, 84, 126]:   # ~2m, 3m, 4m, 6m
    for tn in [1, 2, 3]:
        grid.append({"lookback": lb, "top_n": tn, "sharpe": run_rotation_bt(px, lb, tn)})

grid_df = pd.DataFrame(grid).sort_values("sharpe", ascending=False)
grid_df


## 8) Train/Test split (out-of-sample)

🔍 8) Train/Test Split – Out-of-Sample Validation

Here, we check if the strategy generalizes by testing it on unseen data:

The price data is split at December 2016 into a training period and a testing (out-of-sample) period.

We build and backtest the same momentum strategy on each subset separately.

Key performance metrics are compared — Total Return, CAGR, Sharpe Ratio, Max Drawdown — for both periods.

If performance stays reasonably consistent between Train and Test, it suggests your strategy captures a real effect rather than noise or overfitting.

(You can toggle commissions on/off with use_commissions=True for realism once the runs are stable.)

In [ ]:
# Simple commission model (set to 0 for safety if needed)
def bps_commissions(qty, price, bps=10):
    return abs(qty) * price * (bps / 10_000)

def backtest_with_weights(px_df, lookback_days, top_n, use_commissions=True):
    w = make_rotation_weights(px_df, lookback_days, top_n)
    strat = bt.Strategy(
        "MomentumRotation",
        [bt.algos.RunMonthly(), bt.algos.WeighTarget(w), bt.algos.Rebalance()]
    )
    commissions_fn = (lambda q, p: bps_commissions(q, p, bps=10)) if use_commissions else None
    bt_test = bt.Backtest(
        strat, px_df, initial_capital=INIT_CASH,
        commissions=commissions_fn, progress_bar=False
    )
    res = bt.run(bt_test)
    view = res.stats.loc[['total_return', 'cagr', 'daily_sharpe', 'max_drawdown']]
    return res, view

split_date = "2016-12-31"
px_train = px.loc[:split_date]
px_test  = px.loc[split_date:]

# ✔️ Start with commissions off for stability; flip to True once it's working
res_train, train_stats = backtest_with_weights(px_train, LOOKBACK_DAYS, TOP_N, use_commissions=False)
res_test,  test_stats  = backtest_with_weights(px_test,  LOOKBACK_DAYS, TOP_N, use_commissions=False)

print("=== TRAIN ===")
display(train_stats)
print("=== TEST ===")
display(test_stats)

# Optional: turn commissions on and re-run just once for your chosen params
# res_train_c, train_stats_c = backtest_with_weights(px_train, LOOKBACK_DAYS, TOP_N, use_commissions=True)
# res_test_c,  test_stats_c  = backtest_with_weights(px_test,  LOOKBACK_DAYS, TOP_N, use_commissions=True)
# display(train_stats_c, test_stats_c)
# res_train.plot(title="Train Equity Curve")
# res_test.plot(title="Test Equity Curve")


In [ ]:
import bt

# --- Optional: commissions toggle (set to None to keep it simple) ---
def bps_commissions(qty, price, bps=10):
    # One-time cost per trade; 10 bps = 0.001
    return abs(qty) * price * (bps / 10_000)

use_commissions = False
commissions_fn = (lambda q, p: bps_commissions(q, p, bps=10)) if use_commissions else None

# --- Helper to rebuild our rotation backtest with current params ---
def build_rotation_backtest(px_df, lookback_days, top_n, name="MomentumRotation", commissions_fn=None):
    w = make_rotation_weights(px_df, lookback_days, top_n)
    strat = bt.Strategy(
        name,
        [bt.algos.RunMonthly(), bt.algos.WeighTarget(w), bt.algos.Rebalance()]
    )
    return bt.Backtest(
        strat, px_df, initial_capital=INIT_CASH,
        commissions=commissions_fn, progress_bar=False
    )

# --- Buy & hold helpers ---
def buy_and_hold_backtest(px_df, ticker, commissions_fn=None):
    # Buy once at start, then sit tight
    strat = bt.Strategy(
        f"BH_{ticker}",
        [bt.algos.RunOnce(),
         bt.algos.SelectThese([ticker]),
         bt.algos.WeighEqually(),
         bt.algos.Rebalance()]
    )
    return bt.Backtest(
        strat, px_df, initial_capital=INIT_CASH,
        commissions=commissions_fn, progress_bar=False
    )

def buy_and_hold_equal_weight(px_df, commissions_fn=None):
    strat = bt.Strategy(
        "BH_EQW",
        [bt.algos.RunOnce(),
         bt.algos.SelectAll(),
         bt.algos.WeighEqually(),
         bt.algos.Rebalance()]
    )
    return bt.Backtest(
        strat, px_df, initial_capital=INIT_CASH,
        commissions=commissions_fn, progress_bar=False
    )

def sixty_forty_backtest(px_df, commissions_fn=None):
    # Monthly rebalanced 60% SPY / 40% TLT
    strat = bt.Strategy(
        "BH_60_40",
        [
            bt.algos.RunMonthly(),               # rebalance monthly
            bt.algos.SelectThese(["SPY", "TLT"]),
            bt.algos.WeighSpecified(SPY=0.6, TLT=0.4),
            bt.algos.Rebalance()
        ]
    )
    return bt.Backtest(
        strat, px_df, initial_capital=INIT_CASH,
        commissions=commissions_fn, progress_bar=False
    )

# --- Build backtests ---
bkt_rotation   = build_rotation_backtest(px, LOOKBACK_DAYS, TOP_N, commissions_fn=commissions_fn)
bkt_bh_eqw     = buy_and_hold_equal_weight(px, commissions_fn=commissions_fn)
bkt_benchmarks = [buy_and_hold_backtest(px, t, commissions_fn=commissions_fn) for t in TICKERS]
bkt_60_40 = sixty_forty_backtest(px, commissions_fn=commissions_fn)

# --- Run all together and plot ---
res_cmp = bt.run(bkt_rotation, bkt_bh_eqw, bkt_60_40, *bkt_benchmarks)

# Equity curves (normalized to initial capital). Interactive in VS Code/Jupyter.
res_cmp.plot(title="Momentum Rotation vs Buy-and-Hold Benchmarks")

# Compact stats view for each strategy
try:
    # Some bt versions support .display(); if not, fall back to stats table
    res_cmp.display()
except Exception:
    stats_tbl = res_cmp.stats  # one row per metric; columns per strategy
    display(stats_tbl.loc[['total_return', 'cagr', 'daily_sharpe', 'max_drawdown']])


Sharpe Ratio = E(return)/Volatility
SR = mu / sigma


## 9) Exercises (CFA L2 tie‑ins)

1. **Risk‑parity within winners:** Compute 20‑day volatility per ETF and weight winners ∝ 1/vol instead of equal weight.  
2. **Drawdown control:** If composite momentum (avg of winners’ momentum) < 0, rotate to `TLT` or cash.  
3. **Cost sensitivity:** Re-run with `FEES = 0.0025`, `SLIPPAGE = 0.001` and compare Sharpe.  
4. **Benchmark attribution:** Compare against a **60/40 (SPY/TLT)** benchmark: active return, active risk, information ratio.  
5. **Universe variants:** Add sector ETFs (XLK, XLF, XLV) or credit (LQD, HYG) and re‑test.


1. Org. top 3 3M lookback:
    - sharpe:
    - CAGR:

2. Risk parity top 3 3M Lookback:
3. Risk parity top 3 6M Lookback:
      cagr	0.119053
      daily_sharpe	0.900316
      max_drawdown	-0.262338
4. Risk parity top 3 6M Lookback w/ defense:
      total_return	9.02364
      cagr	0.12315
      daily_sharpe	0.906965
      max_drawdown	-0.354734









## Q3 Cost Sensitivity
*   4.
total_return	7.620296
cagr	0.114647
daily_sharpe	0.852019
max_drawdown	-0.361864




## Q4 benchmark attribution
total_return	3.921744
cagr	0.083611
daily_sharpe	0.78803
max_drawdown	-0.311242

In [ ]:
prices = res_cmp.prices        # DataFrame: columns = strategy names
rets = prices.pct_change().dropna()

port_name  = "MomentumRotation"   # or whatever you called it
bench_name = "BH_60_40"

port_rets  = rets[port_name]
bench_rets = rets[bench_name]
active_rets = port_rets - bench_rets
active_rets.tail(5)


In [ ]:
ANN = 252  # trading days

# Approximate annualized active return
active_return_ann = active_rets.mean() * ANN

# Annualized active risk (tracking error)
active_risk_ann = active_rets.std() * (ANN ** 0.5)
active_return_ann_exact = (1 + active_rets).prod() ** (ANN / len(active_rets)) - 1

info_ratio = active_return_ann / active_risk_ann

print("Active return (ann):", active_return_ann)
print("Active return exact (geometric) (ann):", active_return_ann_exact)
print("Active risk (ann):  ", active_risk_ann)
print("Information ratio:  ", info_ratio)

- Active return: Return over the benchmark

- What is active risk: standard deviation on active return

- What is Information raio: =Active return/active risk
- -> its like Sharpe ratio, but instead of comparing to rfr, you compare to benchmark

In [ ]:
# Plot asset selection
import matplotlib.pyplot as plt

# --- w_all is your daily weight DataFrame ---
# (forward-filled version of w_rebal used for backtesting)

# Make sure columns are in consistent order
w_all = w_all[TICKERS]

plt.figure(figsize=(14, 6))
w_all.plot.area(figsize=(14,6), linewidth=0, alpha=0.8)
plt.title("Portfolio Asset Allocation Over Time")
plt.ylabel("Weight")
plt.xlabel("Date")
plt.legend(loc="upper left", ncol=3)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# --- average weight per ETF across full backtest ---
avg_alloc = w_all.mean() * 100  # in %

# Option 1: Pie chart
plt.figure(figsize=(6, 6))
plt.pie(avg_alloc, labels=avg_alloc.index, autopct="%1.1f%%", startangle=90)
plt.title("Average Portfolio Allocation Over Full Period")
plt.show()

# Option 2: Bar chart (often clearer)
plt.figure(figsize=(8,4))
avg_alloc.sort_values().plot(kind="barh", color="skyblue")
plt.title("Average Portfolio Allocation Over Full Period")
plt.xlabel("Average Weight (%)")
plt.tight_layout()
plt.show()

avg_alloc



In [ ]:
import matplotlib.pyplot as plt

# --- Composite momentum per rebalance date ---
# (average of the momenta of all assets or just winners, depending on your model)
# If you already have mom_on_rebal and w_rebal:
composite_mom = (mom_on_rebal * w_rebal).sum(axis=1)

# Boolean: True when momentum < 0
risk_off = composite_mom < 0

# --- Summary statistics ---
risk_off_ratio = risk_off.mean() * 100
print(f"Composite momentum < 0 in {risk_off_ratio:.1f}% of months "
      f"({risk_off.sum()} out of {len(risk_off)} rebalancing periods).")

# --- Plot ---
fig, ax = plt.subplots(figsize=(12,4))
composite_mom.plot(ax=ax, color="steelblue", lw=1.5)
ax.axhline(0, color="black", lw=1)
ax.fill_between(
    composite_mom.index,
    0, composite_mom,
    where=(composite_mom < 0),
    color="salmon", alpha=0.4, label="Momentum < 0"
)
ax.set_title("Composite Momentum per Rebalance Date")
ax.set_ylabel("3-Month Average Momentum")
ax.legend()
plt.tight_layout()
plt.show()
